In [1]:
%%capture 
!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install matplotlib
!pip install gdal
!pip install numpy

In [7]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona

import matplotlib.pyplot as plt
import matplotlib as mpl
from math import floor

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

In [5]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/jameslab/

dirname = '/content/drive/MyDrive/jameslab/'

Mounted at /content/drive
/content/drive/MyDrive/jameslab


In [3]:
def read_data(fp):

    jack_pine_abundance = gpd.read_file(fp)
    return jack_pine_abundance


def convert_sfile(geometry,t):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''

    new_data= geometry[geometry['VALUE'] == t] #ROUND3 for age
    new_data['dissolvefield'] = 1
    new_data = new_data.dissolve(by='dissolvefield').dissolve()
    new_data['geometry'] = new_data.geometry.buffer(0.000001).unary_union.simplify(1)
    new_data['dissolvefield'] = 1
    new_data = new_data.dissolve(by='dissolvefield').dissolve()

    return new_data

In [10]:

years = list(range(1985,2015+1)) #All years of fires considered

#read the file sent by Doriana and then convert it to epsg 4326
df_origin = read_data('doriana/fire_severity_selected_v003.shp').to_crs('EPSG:4326')
df_harvest = read_data('doriana/harvest_shapefile.shp')
#sort by year to make easier 
df_origin = df_origin.sort_values('YEAR')

#loop through the years 
for y in years:
    print(y)
    try: 
      df1 = read_data('doriana/shp/'+str(y)+'_shp.shp') #open pre-made files in drive 
    except: 
      df1 = read_data('doriana/shp/'+str(y)+'_shp_n.shp') #case of repaired file 
    
    
    harvest = [] #List to collect area 
    names = [] 
    for geom,n in zip(list(df_origin['geometry']),list(df_origin['FIRE_ID'])): #Loop through each fire polygon 

        clip_harvest = df1.clip(geom,keep_geom_type=True) #Make sure clip does not produce lines 
        clip_harvest = clip_harvest.clip(df_harvest,keep_geom_type=True)
        if len(list(clip_harvest['geometry'])) > 0: #If there is intersection b/t fire & disturbance area 
          print(n)
          geom2 = clip_harvest.to_crs('ESRI:102001') #convert to projected system 

          harvest.append(geom2.area[0]) #Calc area and append to list 
          names.append(n)
        else: 
          harvest.append(0) #If area disturbed is 0 in that year 
          names.append(n)

    df_origin[str(y)+'_AreaDisturbed'] = harvest #Create the new column and append data 

df_origin.to_file('doriana/area_disturbed_cliph.geojson',driver="GeoJSON") #send geojson to drive 

1985
2001-N20157
2004-R90086
2004-K70172
2006-G70329
2006-K21065
2009-C50418
2009-V11183
1986
1985-N50075
1985-N50059
1986-G90013
2004-R90086
2004-K70172
2006-G60418
2006-K21065
2009-C50418
2009-V11183
2015-G90175
1987
1985-N50059
1986-G90013
2004-K70172
2006-G60418
2006-G70329
2006-K21065
2015-G90175
1988
1994-N20025
2004-K70172
2006-G70329
2006-G70309
2007-N50229
2009-C50418
2014-N20399
2015-G90175
1989
1994-K20062
1994-N20025
2003-N70454
2006-G60418
2009-C40573
2009-C50402
2014-N20399
2015-G90175
1990
1985-N50059
1990-VA0032
1990-V30102
2001-N40186
2004-VA0194
2004-R10243
2009-C50418
1991
1990-VA0032
1990-V30102
2001-N40186
2004-R10243
2006-G80343
2009-C50418
1992
1990-V30102
1994-K60060
1994-K20062
2003-K30465
2004-VA0194
2006-G80343
2006-G70329
2006-G70309
2009-C50418
1993
1993-G90016
1994-K60060
2003-N70454
2004-K10398
2009-C50418
2010-C10225
1994
1994-K60065
1994-K60060
1994-K20062
1994-N20025
2003-N70454
2004-K10398
2004-N40379
2006-G60418
2007-N50229
2009-C50418
2009-V11183
20

ERROR:fiona._env:doriana/shp/2004_shp.shp: No such file or directory


2015-G90175
2004
1985-N50075
1990-V30102
1994-K20062
1998-N30235
2003-N50546
2003-N40719
2003-N70454
2003-N40198
2003-K40695
2003-K30465
2003-K10612
2003-C50199
2003-N40570
2004-R90086
2004-VA0194
2004-R90101
2004-K10398
2004-R10243
2004-N40379
2004-C50114
2004-R20052
2004-K70172
2006-C50148
2006-G60237
2009-C50418
2009-V11062
2010-C10225
2012-G90303


ERROR:fiona._env:doriana/shp/2005_shp.shp: No such file or directory


2005
1985-N50075
2003-N50546
2003-N40719
2003-N70454
2003-K30465
2003-K10612
2003-K70611
2003-N40570
2004-R90086
2004-VA0194
2004-K10398
2004-R10243
2004-N40379
2004-C50114
2004-R20052
2004-K10413
2004-K10426
2004-K70172
2004-K70498
2005-G40110
2006-G80349
2006-C50148
2006-K21065
2009-C50418
2009-V11183
2009-V11062
2010-C10225
2012-G90303


...

2013
2001-N20479
2003-K30465
2004-C50149
2006-G80353
2006-G70329
2006-G70309
2006-C50148
2007-N50229
2007-C40139
2009-C50418
2009-C50489
2012-G90325
2012-G90303
2013-G90114
2013-G90119
2014-N20399


ERROR:fiona._env:doriana/shp/2014_shp.shp: No such file or directory


2014
1985-N50059
2003-N40719
2003-K30465
2004-R90086
2004-R90101
2004-R10243
2004-R20052
2006-G80353
2006-G70309
2006-G60439
2006-C50148
2006-G60237
2009-C50418
2009-V11062
2012-G90303
2013-G90119
2013-R10252
2014-N20399


ERROR:fiona._env:doriana/shp/2015_shp.shp: No such file or directory


2015
1985-N50059
1986-G90013
1994-K20062
1998-R90068
2003-N70454
2003-C50199
2004-R90086
2004-R10243
2004-R20052
2004-K70172
2006-G80349
2006-G80353
2006-G80343
2006-G60418
2006-G70329
2006-G70309
2006-G60439
2006-G40362
2006-G60237
2007-N50229
2008-G90200
2009-C40573
2009-C50418
2009-G60414
2009-V11062
2012-G90303
2013-G90119
2013-R10252
2014-K70430
2014-N20399
2015-G90175
